# DistilBertForSequenceClassification

In [1]:
import pandas as pd
import os
import re
import emoji
from transformers import Trainer, TrainingArguments, AdamW
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score

In [3]:
chunks = pd.read_csv("merged_emoji_data.csv", chunksize=1000)  # read in chunks of 1000 rows since there is a lot of data
final_merged_df = pd.concat(chunks, ignore_index=True)  # combine the chunks

In [4]:
# change the text, title, and ucode_short to type string
final_merged_df['text'] = final_merged_df['text'].astype('string')
final_merged_df['title'] = final_merged_df['title'].astype('string')
final_merged_df['ucode_short'] = final_merged_df['ucode_short'].astype('string')

# drop all the NA values 
final_merged_df = final_merged_df.dropna()

In [5]:
final_merged_df

,text,title,ucode_short,ucode,cleaned_text
0,Text,egg,1f95a,🥚,text
1,Happy Easter😀🐇🥚,egg,1f95a,🥚,happy easter
2,@elonmusk @teslaownersSV #eggs Easter eggs tod...,egg,1f95a,🥚,eggs easter eggs today will surely be deliciou...
3,We hope everyone has a Hoppy Easter! 😉🐰🥚🐣🌷 #H...,egg,1f95a,🥚,we hope everyone has a hoppy easter happyeaste...
4,🐰🥚 Get ready to hop into the most egg-citing d...,egg,1f95a,🥚,get ready to hop into the most eggciting digit...
...,...,...,...,...,...
848550,Happy Easter 🐣 💓 from my brother ❤️ Armen Grig...,hatching chick,1f423,🐣,happy easter from my brother armen grigoryan s...
848551,"Have a great finish to your weekend, Happy Eas...",hatching chick,1f423,🐣,have a great finish to your weekend happy east...
848552,"Beef ribs, smothered turkey wings, cheesy pota...",hatching chick,1f423,🐣,beef ribs smothered turkey wings cheesy potato...
848553,Happy Easter! Hope you all have an amazing day...,hatching chick,1f423,🐣,happy easter hope you all have an amazing day ...


In [6]:
label_encoder = LabelEncoder()
final_merged_df['emoji_label'] = label_encoder.fit_transform(final_merged_df['ucode'])

sample_size = 75000  # based on your testing

data_sampled, _ = train_test_split(
    final_merged_df,
    train_size=sample_size,
    stratify=final_merged_df['emoji_label'],  
    random_state=42
)
# # shuffle the reduced dataset
data_sampled = data_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into training and testing sets
X = data_sampled['cleaned_text']
y = data_sampled['emoji_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
X_train = X_train.astype(str).tolist()  # Convert to string type and then to list
X_test = X_test.astype(str).tolist()

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# load DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# tokenize data
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

# define number of classes
num_classes = len(label_encoder.classes_)

## continue to train

In [9]:
# load DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_classes).to(device)

# dataset class
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Prepare datasets
train_dataset = CustomDataset(
    input_ids=X_train_tokens['input_ids'],
    attention_mask=X_train_tokens['attention_mask'],
    labels=torch.tensor(y_train.values)
)

eval_dataset = CustomDataset(
    input_ids=X_test_tokens['input_ids'],
    attention_mask=X_test_tokens['attention_mask'],
    labels=torch.tensor(y_test.values)
)

# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=False,
    report_to="none"
)

# define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# train model
trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/esthercho/anaconda3/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,3.061000,3.077321,0.175200
2,2.864800,3.029427,0.202200
3,2.163600,3.152775,0.202000


TrainOutput(global_step=22500, training_loss=2.8009784339057076, metrics={'train_runtime': 11137.8385, 'train_samples_per_second': 16.161, 'train_steps_per_second': 2.02, 'total_flos': 4800190435920000.0, 'train_loss': 2.8009784339057076, 'epoch': 3.0})

In [10]:
import emoji
def clean_text(text):
    # convert to lowercase
    text = text.lower()
    # remove all mentions (e.g., @username)
    text = re.sub(r'@[\w]+', '', text)  # removes anything starting with @
    # remove emojis if not needed (optional, you can skip this part if emojis should be kept)
    text = emoji.replace_emoji(text, replace='')  # this removes all emojis

    # remove non-alphanumeric characters (keep letters and numbers)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [11]:
import emoji
# Define the input texts
new_texts = ["I'm anxious about class. Everything is stressing me out.", "Easter egg hunt this sunday!", "Good job on the promotion! Im'm very proud of you."]

# Clean and tokenize the texts
new_texts_cleaned = [clean_text(text) for text in new_texts]
new_tokens = tokenizer(
    new_texts_cleaned,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=512
)

# Move model and tokens to the appropriate device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = model.to(device)
new_tokens = {key: val.to(device) for key, val in new_tokens.items()}

# Get predictions from the model
model.eval()
with torch.no_grad():
    outputs = model(**new_tokens)
    logits = outputs.logits

# Decode the predicted labels
predicted_labels = torch.argmax(logits, dim=-1)
predicted_emoji = label_encoder.inverse_transform(predicted_labels.cpu().numpy())

# Display the predicted emojis for each input text
for text, emoji in zip(new_texts, predicted_emoji):
    print(f"Text: {text} -> Predicted Emoji: {emoji}")

Text: I'm anxious about class. Everything is stressing me out. -> Predicted Emoji: 😢
Text: Easter egg hunt this sunday! -> Predicted Emoji: 🥚
Text: Good job on the promotion! Im'm very proud of you. -> Predicted Emoji: 😊


In [12]:
# Save the model (commented out so it doesn't save every time
# trainer.save_model("./DistilBert_model")

In [9]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = DistilBertForSequenceClassification.from_pretrained("./DistilBert_model").to(device)

In [11]:
import emoji
# define the input texts
new_texts = ["Its a sunny day today!", 
             "I'm anxious about class. Everything is stressing me out", 
             "Easter egg hunt this sunday!", 
             "Good job on the promotion! Im'm very proud of you.",
            "Happy birthday amanda!", 
             "Nice work on the project!", 
             "Did you feed the bunny?",
             "I like cheese",
             "Woke brain virus",
            "Nick denies being gay",
            "I don't think ill be able to finish my project today"]

# clean and tokenize the texts
new_texts_cleaned = [clean_text(text) for text in new_texts]
new_tokens = tokenizer(
    new_texts_cleaned,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=512
)

# move model and tokens to the appropriate device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = model.to(device)
new_tokens = {key: val.to(device) for key, val in new_tokens.items()}

# get predictions from the model
model.eval()
with torch.no_grad():
    outputs = model(**new_tokens)
    logits = outputs.logits

# decode the predicted labels
predicted_labels = torch.argmax(logits, dim=-1)
predicted_emoji = label_encoder.inverse_transform(predicted_labels.cpu().numpy())

# display the predicted emojis for each input text
for text, emoji in zip(new_texts, predicted_emoji):
    print(f"Text: {text} -> Predicted Emoji: {emoji}")

Text: Its a sunny day today! -> Predicted Emoji: ☀
Text: I'm anxious about class. Everything is stressing me out -> Predicted Emoji: 😢
Text: Easter egg hunt this sunday! -> Predicted Emoji: 🥚
Text: Good job on the promotion! Im'm very proud of you. -> Predicted Emoji: 😊
Text: Happy birthday amanda! -> Predicted Emoji: 🎉
Text: Nice work on the project! -> Predicted Emoji: 👍
Text: Did you feed the bunny? -> Predicted Emoji: 🐰
Text: I like cheese -> Predicted Emoji: 😋
Text: Woke brain virus -> Predicted Emoji: 😨
Text: Nick denies being gay -> Predicted Emoji: 💀
Text: I don't think ill be able to finish my project today -> Predicted Emoji: 😢
